## 从0开始构建一个简单的神经网络
### 神经网络（Neural Network）


### 关于这个Notebook

在这个Notebook中，我们将学习如何使用Pytorch框架来构建简单的全连接神经网络，以及使用该网络对手写数字进行识别。
首先，我们安装pytorch框架：

In [ ]:
!pip install torch

导入我们需要的包
Torchvision包括很多流行的数据集、模型架构和用于计算机视觉的常见图像转换模块，它是PyTorch项目的一部分。

In [2]:
import torch
import torchvision
from matplotlib import pyplot as plt
import torch.nn as nn
import torch.nn.functional as F

### mnist数据集介绍

这个数据集包含 70,000 张手写体数字的灰度图(0=黑, 255=白)，这些图每一张的尺寸都是 28 像素乘以 28 像素，共 784 个像素，按照图像的标准来说，它们的尺寸非常小。这些图中的数字都是居中的，并且四边都留空了。这使得手写体识别可以比较方便地用标准的全连接网络来解决。
下载数据集，并且构建dataset.

In [6]:
path = './data/'
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),
                                torchvision.transforms.Normalize(mean = [0.5],std = [0.5])])
trainData = torchvision.datasets.MNIST(path,train = True,transform = transform,download = True)
testData = torchvision.datasets.MNIST(path,train = False,transform = transform)

In [ ]:
image, label = trainData[0]
print(image.shape) # 打印图像尺寸 (1, 28, 28)
print(label) # 打印标签
img = image.numpy().transpose(1, 2, 0)
plt.imshow(img)

在训练之前，我们需要指定训练设备，构建dataloader：

1.指定训练设备,device：将张量部署在指定的运算设备上进行计算

2.构建dataloader，*DataLoader*是 PyTorch 提供的一个数据加载器，用于对数据进行批量加载和处理。它在训练神经网络时起到了重要的作用。DataLoader 可以自动完成数据的批量加载、随机洗牌（shuffle）、并发预取等操作，从而提高模型训练的效率。

In [10]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 256 

trainloader = torch.utils.data.DataLoader(dataset = trainData,batch_size = BATCH_SIZE,shuffle = True)#shuffle = True 意味着会对数据集打乱
testloader = torch.utils.data.DataLoader(dataset = testData,batch_size = BATCH_SIZE)
print(device)

cuda:0


### 搭建神经网络
首先搭建最基本的全连接3层网络，输入层 784个input，第一个隐藏层包含64个神经元，relu函数作为激活函数，第二个隐藏层包含64个神经元，relu函数作为激活函数，输出层10个神经元，softmax作为激活函数(但是请注意，这一层激活函数是包含在了损失函数里面，网络中没有这一层)。

In [11]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()

        #两个全连接的隐藏层，一个输出层
        #因为图片是28*28的，需要全部展开，最终我们要输出数字，一共10个数字。
        #10个数字实际上是10个类别，输出是概率分布，最后选取概率最大的作为预测值输出
        hidden_1 = 64
        hidden_2 = 64
        self.fc1 = nn.Linear(28 * 28,hidden_1)
        self.fc2 = nn.Linear(hidden_1,hidden_2)
        self.fc3 = nn.Linear(hidden_2,10)
        #self.dropout = nn.Dropout(0.2),防止过拟合

    def forward(self,x):
        x = x.view(-1,28 * 28)
        x = F.relu(self.fc1(x))
        #x = self.dropout(x)
        x = F.relu(self.fc2(x))
        #x = self.dropout(x)
        x = self.fc3(x)
        return x
    
model = Net().to(device)
model

Net(
  (fc1): Linear(in_features=784, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=10, bias=True)
)

In [ ]:
t2 = torch.rand(28,28)
print(t2.dtype)
t = t2.to(device)
model.eval()
y = model(t)
print(y)
y = torch.nn.functional.softmax(y,dim=1)    
print(y)
y.argmax()


In [ ]:
for i in model.parameters():
    print(i.shape)

In [12]:
#定义损失函数：这里采用交叉熵损失函数
#优化器：这里选用RMSprop 也可以采用（SGD、Adam）
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(params = model.parameters(),lr = 0.01)

In [13]:
#训练
n_epochs = 2

for epoch in range(0,n_epochs):
    train_loss = 0.0
    train_loss_ = 0.0
  
    model.train() # 开启训练模式，参数可更新
    for data,target in trainloader:
        data,target = data.to(device), target.to(device)
       # print(data.size(0))
        optimizer.zero_grad() # 每一步都需要将梯度归零，防止累计
        output = model(data)#得到预测值

        loss = criterion(output,target)
        loss.backward()  #反向传播，计算梯度

        optimizer.step() #使用梯度对网络参数进行更新
        train_loss += loss.item()*data.size(0)
    train_loss = train_loss / len(trainloader.dataset)
    print('Epoch:  {}  \tTraining Loss: {:.6f}'.format(epoch + 1,train_loss))
  
    model.eval() #开启评估模式，参数不可更新,不进行反省传播，当网络中存在dropout和bn的时候，一定要加上。
    print(optimizer.param_groups[0]['lr'])
    correct = 0
    for data, target in testloader:
        data,target = data.to(device), target.to(device)
        output = model(data)
        out = torch.argmax(output, dim=1)
        correct += torch.sum(out == target)
  
    testAccuracy = correct / len(testloader.dataset)
    print('Epoch:  {}  \tTest Accuracy: {:.6f}'.format(epoch + 1,testAccuracy))


Epoch:  1  	Training Loss: 1.958830
0.01
Epoch:  1  	Test Accuracy: 0.677600
Epoch:  2  	Training Loss: 0.934634
0.01
Epoch:  2  	Test Accuracy: 0.836100


通过分析每轮训练出来的loss及accuracy变化，分析模型误差的原因，是过拟合还是欠拟合，从而采取相应的措施进行优化，例如：过拟合可以增加dropout比例、增加数据量或者正则化等方法。欠拟合，可以增加模型复杂度、优化学习率，增加特征数量等。
### 模型的保存及测试
可以使用model.load(filepath)将模型和权重保存在一个文件中，主要有两种方法：

1.只保存参数；
2.保存整个模型 (结构+参数)。


In [14]:
#保存整个model的状态
torch.save(model, 'model0812.pth')

### 推理过程
可以使用model.load(filepath)来重新加载你的模型


In [17]:
import torch
import numpy as np
import cv2
import torchvision
model_predict = torch.load("model0812.pth").to("cpu")


In [18]:
data = cv2.imread('./testpic/2_2.png')
data = cv2.cvtColor(data,cv2.COLOR_BGR2GRAY)

data_normal = cv2.resize(data,(28,28),cv2.INTER_LINEAR)

input_data = transform(data_normal)
print(input_data.shape)
output = model_predict(input_data)

print(output)

_, predicted = torch.max(output, 1)
print(_,predicted)
print('Predicted class: ', predicted.item())#item 单个元素 转为标量

torch.Size([1, 28, 28])
tensor([[-6.8162, -0.4700,  8.8663, -3.2106, -1.7154, -0.4724,  6.2760, -4.5514,
         -4.4659, -3.1768]], grad_fn=<AddmmBackward0>)
tensor([8.8663], grad_fn=<MaxBackward0>) tensor([2])
Predicted class:  2


### 了解损失函数

In [ ]:
import sympy
x = sympy.symbols('x')
sigma = 1/(1 + sympy.exp(-x))
tanh = (sympy.exp(x) - sympy.exp(-x)) / (sympy.exp(x) + sympy.exp(-x))
dsigma = sigma.diff(x)
dtanh = tanh.diff(x)
sympy.plot(dtanh)
sympy.plot(dsigma)